In [30]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re
cache = None
api = None
def _download():
    global cache, api
    if api is None:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    cache = api.download_issues(project_id="pbssoftware")
def download(force=False):
    global cache
    if cache is None or force:
        _download()

In [3]:
def relevant(i):
    if i.sprint_milestone != "3.0" or i.tracker != "Specification":
        return False
    if i.status in ("Rejected", "Closed"):
        return False
    return True

def filter_issues():
    global cache, issues_30
    issues_30 = [i for i in cache.values() if relevant(i)]
    print("Filtered %d relevant issues" % len(issues_30))
    
def get_issues(recache=False):
    download(recache)
    filter_issues()
    return issues_30.copy()

In [15]:
for i in get_issues():
    exp = "*+FRS%d.1+*" % i.id
    d = i.description
    if d.find(exp) == -1:
        print(i.id)

3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
989
3083
3087
3090
3093
3101
3102
3106
3107
3108
3109
3114
3118
1885
3120
3122
3124
3130
3131
3132
3133
3134
3135
3136
2115
3141
3145
2231
3150
3152
3153
3154
3155
3156
3157
3158
3159
3161
3163
3164
3165
3166
3167
3168
3175
3176
3177
3178
3183
3186
3188
3189
3191
3192
3194
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
1902
3223
3224
3225
3226
3229
3230
3231
3233
3234
3235
3236
3237
3238
3240
3242
3244
3245
3246
3247
3248
3249
3250
3252
3253
3254
3255
3258
3259
3260
3261
3264
3266
3267
3268
3269
3272
2251
3280
3287
3288
3289
3290
1262
2303
2304
2309
2316
2320
2321
1953
2367
396
2394
2395
2404
2415
2445
2458
1263
2460
2230
420
2469
2476
2484
1583
2500
2502
2516
2522
1073
2592
2596
2626
2031
2660
2669
2681
2684
2703
2711
2712
2713
2715
2716
1694
2725
2726
2733
2734
2735
2747
2752
2757
2758
2767
2769
1749
1754
2780
1759
1572
2787
2789
2794
2796
2801
2805
1783
2808
2809
2810
2812
2814
281

In [33]:
frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
for i in get_issues():
    if not frs_re.search(i.description):
        if i.id == 3194:  # URS issue
            continue 
        if web_frs_re.search(i.description):
            print("Goddamit Christian: %d" % i.id)
        else:
            print("Goddamnit Nathan: %d" % i.id)

In [36]:
frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
n=0
noncompliant = []; add = noncompliant.append
for i in get_issues():
    exp = "*+FRS%d.1+*" % i.id
    exp2 = "*+FRS%d.0+*" % i.id
    if i.description.find(exp) == -1 and i.description.find(exp2) == -1:
        if not web_frs_re.search(i.description):
            add("Goddamnit Nathan: %d" % i.id)
        
print("Found %d noncompliant issues" % len(noncompliant))
for msg in noncompliant:
    print(msg)

Found 80 noncompliant issues
Goddamnit Nathan: 989
Goddamnit Nathan: 2115
Goddamnit Nathan: 3176
Goddamnit Nathan: 3194
Goddamnit Nathan: 1902
Goddamnit Nathan: 3225
Goddamnit Nathan: 3226
Goddamnit Nathan: 3288
Goddamnit Nathan: 3289
Goddamnit Nathan: 1262
Goddamnit Nathan: 2304
Goddamnit Nathan: 2316
Goddamnit Nathan: 2320
Goddamnit Nathan: 2321
Goddamnit Nathan: 2367
Goddamnit Nathan: 2404
Goddamnit Nathan: 2445
Goddamnit Nathan: 2458
Goddamnit Nathan: 1263
Goddamnit Nathan: 2460
Goddamnit Nathan: 420
Goddamnit Nathan: 2476
Goddamnit Nathan: 1583
Goddamnit Nathan: 2502
Goddamnit Nathan: 2516
Goddamnit Nathan: 2522
Goddamnit Nathan: 1073
Goddamnit Nathan: 2596
Goddamnit Nathan: 2626
Goddamnit Nathan: 2734
Goddamnit Nathan: 2757
Goddamnit Nathan: 2769
Goddamnit Nathan: 1754
Goddamnit Nathan: 2780
Goddamnit Nathan: 1759
Goddamnit Nathan: 2787
Goddamnit Nathan: 2796
Goddamnit Nathan: 2805
Goddamnit Nathan: 2808
Goddamnit Nathan: 2819
Goddamnit Nathan: 2825
Goddamnit Nathan: 2828
Goddamn

In [6]:
def search(recache=False):
    compliant_search = re.compile(r"[\+\*]{1,2}(?:3\.0Web)?FRS[\d\.]*\:?[\+\*]{1,2}").search
    frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
    web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
    n=0
    def compliant(i):
        return i.id == 3194 or compliant_search(i.description)
    noncompliant = []; add = noncompliant.append
    for i in get_issues(recache):
        if not compliant(i):
            #if not web_frs_re.search(i.description):
                add("Goddamnit Nathan: %d" % i.id)

    print("Found %d noncompliant issues" % len(noncompliant))
    for msg in noncompliant:
        print(msg)

In [7]:
search()

Filtered 81 relevant issues
Found 0 noncompliant issues


In [31]:
compliant_search = re.compile(r"^[\+\*]{1,2}(?:3\.0Web)?FRS\d+\:?[\+\*]{1,2}", re.MULTILINE).search
frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")

def compliant_description(d):
    return compliant_search(d) is not None
def compliant(i):
    return (i.id == 3194 or compliant_description(i.description))

def search(recache=False):
    compliant_search = re.compile(r"^[\+\*]{1,2}(?:3\.0Web)?FRS\d+\:?[\+\*]{1,2}").search
    frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
    web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
    n=0
    noncompliant = []
    def add(id, msg):
        noncompliant.append((id, msg))
    for i in get_issues(recache):
        if not compliant(i):
            #if not web_frs_re.search(i.description):
                add(i.id, "Goddamnit Nathan: %d" % i.id)

    print("Found %d noncompliant issues" % len(noncompliant))
    noncompliant.sort(key=lambda arg:arg[0])
    for _, msg in noncompliant:
        print(msg)
search(True)

Filtered 80 relevant issues
Found 62 noncompliant issues
Goddamnit Nathan: 1583
Goddamnit Nathan: 1902
Goddamnit Nathan: 2115
Goddamnit Nathan: 2210
Goddamnit Nathan: 2304
Goddamnit Nathan: 2316
Goddamnit Nathan: 2320
Goddamnit Nathan: 2321
Goddamnit Nathan: 2367
Goddamnit Nathan: 2404
Goddamnit Nathan: 2445
Goddamnit Nathan: 2458
Goddamnit Nathan: 2460
Goddamnit Nathan: 2476
Goddamnit Nathan: 2502
Goddamnit Nathan: 2516
Goddamnit Nathan: 2522
Goddamnit Nathan: 2596
Goddamnit Nathan: 2626
Goddamnit Nathan: 2734
Goddamnit Nathan: 2757
Goddamnit Nathan: 2769
Goddamnit Nathan: 2780
Goddamnit Nathan: 2787
Goddamnit Nathan: 2796
Goddamnit Nathan: 2805
Goddamnit Nathan: 2819
Goddamnit Nathan: 2825
Goddamnit Nathan: 2828
Goddamnit Nathan: 2851
Goddamnit Nathan: 2856
Goddamnit Nathan: 2859
Goddamnit Nathan: 2860
Goddamnit Nathan: 2864
Goddamnit Nathan: 2865
Goddamnit Nathan: 2871
Goddamnit Nathan: 2875
Goddamnit Nathan: 2876
Goddamnit Nathan: 2880
Goddamnit Nathan: 2881
Goddamnit Nathan: 2884


In [33]:
import clipboard
print(compliant_description(clipboard.paste()))

True
